In [1]:
import numpy as np
import rforest
import sys
import random
import data_preparation as dataPrep
import matplotlib.pyplot as plt

In [2]:
Ntr = 0.8
N_ftr = 5
depth = N_ftr + 1
Nbag = 200
Nlabel = 2
Ntotal = 1

In [3]:
def calculate_error(output):
    n_examples = output.shape[0]
    prediction = output[:, N_ftr]
    target = output[:, N_ftr + 1]
    misclass = np.where(target == prediction, 0, 1)
    error = np.mean(misclass)
    precision = 1. - error
    return error, precision

In [4]:
np.random.seed(1)
random.seed(1)

In [5]:
data = dataPrep.titanic('train.csv', LABELED=True)
N0 = int(Ntr * np.asarray(data[0]).shape[0])
N1 = int(Ntr * np.asarray(data[1]).shape[0])
print N0, N1

439 273


In [6]:
trainData  = [data[0][:N0, :], data[1][:N1, :]]
print np.asarray(trainData[0]).shape, np.asarray(trainData[1]).shape

(439, 6) (273, 6)


In [7]:
crossValData = [data[0][N0 + 1:, :], data[1][N1 + 1:, :]]
print np.asarray(crossValData[0]).shape, np.asarray(crossValData[1]).shape
print crossValData[1]

(109, 6) (68, 6)
[[0.00000000e+00 1.00000000e+00 4.20000000e+01 2.62875000e+01
  1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 2.20000000e+01 1.51550000e+02
  1.00000000e+00 1.00000000e+00]
 [0.00000000e+00 3.00000000e+00 2.96991176e+01 1.52458000e+01
  3.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 2.40000000e+01 4.95042000e+01
  1.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00 4.80000000e+01 5.20000000e+01
  2.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 3.80000000e+01 2.27525000e+02
  1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 2.00000000e+00 2.70000000e+01 1.05000000e+01
  1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 2.00000000e+00 6.00000000e+00 3.30000000e+01
  2.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00 2.70000000e+01 5.31000000e+01
  2.00000000e+00 1.00000000e+00]
 [1.00000000e+00 2.00000000e+00 3.00000000e+01 2.10000000e+01
  4.00000000e+00 1.00000000e+00]
 [1.00000000e+00 3.00000000e+00 2

In [8]:
testData = dataPrep.titanic('test.csv', LABELED=False)

In [9]:
finalSurv = np.zeros(419)

In [10]:
for Ntree in range(Ntotal):
    if Ntree % 10 == 0:
        print Ntree
    #for lbl in trainData:
    #    np.random.shuffle(lbl)
    #idset = np.random.randint(0, Ntr, Nbag)
    #baggedData = [lbl[idset, :] for lbl in trainData]
    baggedData = trainData
    RF = rforest.DecisionTree(depth, 2)
    RF.train([baggedData])

    output = RF.predict_label(baggedData)
    error, precision = calculate_error(output)
    print 'error and precision on training set: {}, {}'.format(
        round(error, 3), round(precision, 3))

    output = RF.predict_label(crossValData)
    print np.asarray(output).shape
    error, precision = calculate_error(output)
    print 'error and precision on validation set: {}, {}\n'.format(
        round(error, 3), round(precision, 3))

    output = RF.predict_label(testData)
    output = zip(output[:, N_ftr], output[:, N_ftr + 1])
    output.sort(key=lambda tup: tup[0])
    pId, survival = zip(*output)

    finalSurv += survival
    print RF.labelVal

0
d entropy 0.21755089758655333
split val 0.0
373 86 66 187
d entropy 0.034156639294141256
split val 1.0
65 36 308 50
d entropy 0.19393713667634316
split val 2.000400160064026
7 125 59 62
d entropy 0.04539905500840313
split val 17.02713085234094
0 3 65 33
d entropy 0.03577558339493492
split val 12.009539815926372
11 11 297 39
d entropy 0.01823972701935017
split val 2.0
1 1 6 124
d entropy 0.04994932385113693
split val 38.01530612244898
50 61 9 1
d entropy -4.808743030260912e-05
np split
d entropy 0.037711249548887715
split val 26.03673805522209
6 0 59 33
d entropy 0.2762027109482458
split val 20.530836974789914
0 5 11 6
d entropy 0.007782419320386591
split val 7.764705882352941
77 5 220 34
d entropy 0.999855752133692
split val 26.0
0 1 1 0
d entropy 0.025868252006896575
split val 28.77389243697479
5 49 1 75
d entropy 0.07418434357955561
split val 24.165126050420167
38 59 12 2
d entropy 0.19348717315483038
split val 9.598619447779111
2 1 7 0
d entropy -2.4044316237472907e-05
np split
d 

In [11]:
finalSurv = 1. * finalSurv / Ntotal
print finalSurv

survival = np.where(finalSurv >= .5, 1, 0)
with open('submit.txt', 'w') as f:
    f.write('PassengerId,Survived\n')
for ii in range(np.shape(pId)[0]):
    if 0 == ii:
        continue
    with open('submit.txt', 'a') as f:
        f.write(str(int(pId[ii])) + ',' + str(int(survival[ii])) + '\n')

[0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1.
 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1.
 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0.
 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0.
 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0.
 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0.
 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0.
 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.